##

In [1]:
def predict(alpha: float, beta: float, x_i: float) -> float:
    return beta * x_i + alpha

def error(alpha: float, beta: float, x_i: float, y_i: float) -> float:
    """Ошибка предсказания beta * x_i + alpha,
       когда фактически значение равно y_i"""
    return predict(alpha, beta, x_i) - y_i

In [2]:
from theory.linear_algebra import Vector

def sum_of_squerrors(alpha: float, beta: float, x: Vector, y: Vector) -> float:
    return sum(error(alpha, beta, x_i, y_i) ** 2
               for x_i, y_i in zip(x, y))

### МНК

In [3]:
from typing import Tuple
from theory.linear_algebra import Vector
from istatistics import correlation, mean, standard_deviation

def least_squares_fit(x: Vector, y: Vector) -> Tuple[float, float]:
    """Учитывая векторы y и x, отыскать
       значения alpha и beta по наименьшим квадратам"""
    beta = correlation(x, y) * standard_deviation(y) / standard_deviation(x)
    alpha = mean(y) - beta * mean(x)
    return alpha, beta

### Тест для модели

In [4]:
x = [i for i in range(-100, 110, 10)]
y = [3 * i - 5 for i in x]

# y = 3x -5
assert least_squares_fit(x, y) == (-5, 3)


In [5]:
from istatistics import num_friends_good, daily_minutes_good

alpha, beta = least_squares_fit(num_friends_good, daily_minutes_good)

assert 22.9 < alpha < 23.0
assert 0.9 < beta < 0.905

### Коэффициент детерминации ($R^2$)

In [13]:
from istatistics import de_mean

def total_sum_of_squares(y: Vector) -> float:
    """Полная сумма квадратов отклонений y_i от их среднего"""
    return sum(v ** 2 for v in de_mean(y))

def r_squared(alpha: float, beta: float, x: Vector, y: Vector) -> float:
    """Доля отклонения в y, улавливаемая моделью, которая равна
       '1 - доля отклонения y, не улавливаемая моделью"""
    return 1.0 - (sum_of_squerrors(alpha, beta, x, y) / 
                  total_sum_of_squares(y))
    
rsq = r_squared(alpha, beta, num_friends_good, daily_minutes_good)
assert 0.328 < rsq < 0.330

### Применение градиентного спуска

In [18]:
import random
import tqdm
from gradient_descent import gradient_step

num_epoch = 10000
random.seed(0)

guess = [random.random(), random.random()]      # Выбрать случайное число для запуска

learning_rate = 0.00001                         # Темп усвоения

with tqdm.trange(num_epoch) as t:
    for _ in t:
        alpha, beta = guess

        # Частная производная потери по отношению к alpha
        grad_a = sum(2 * error(alpha, beta, x_i, y_i)
                     for x_i, y_i in zip(num_friends_good,
                                         daily_minutes_good))
        
        # Частная производна потери по отношению к alpha
        grad_b = sum(2 * error(alpha, beta, x_i, y_i) * x_i
                     for x_i, y_i in zip(num_friends_good,
                                         daily_minutes_good))
        
    # Вычислить потерю для вставки в описание tqdm
    loss = sum_of_squerrors(alpha, beta,
                            num_friends_good, daily_minutes_good)
    t.set_description(f"потеря: {loss:.3f}")

    # В заключение обновить догадку
    guess = gradient_step(guess, [grad_a, grad_b], -learning_rate)

alpha, beta = guess

print(alpha, beta)

100%|██████████| 10000/10000 [00:00<00:00, 15402.36it/s]

0.9382344118777632 1.426212885703206
